In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
tqdm.pandas()

In [2]:
jan_data_path = "./data/yellow_tripdata_2022-01.parquet"
feb_data_path = "./data/yellow_tripdata_2022-02.parquet"

In [3]:
df_jan = pd.read_parquet(jan_data_path)

###   Q1:   Read the data for January. How many columns are there?

In [4]:

print(len(df_jan.columns))

19



### Q2: What's the standard deviation of the trips duration in January?

In [5]:
def get_duration(item):
    timedelta =  item['tpep_dropoff_datetime'] - item['tpep_pickup_datetime']
    n_mins = timedelta.total_seconds() / 60
    return  n_mins

In [6]:
df_jan['duration(mins)'] = df_jan.progress_apply(lambda x:get_duration(x),axis = 1)

100%|██████████| 2463931/2463931 [00:15<00:00, 161275.45it/s]


In [7]:
np.std(df_jan['duration(mins)'])

46.445295712725304

### Q3: What fraction of the records left after you dropped the outliers?

In [8]:
df =df_jan[(df_jan['duration(mins)'] >= 1) & (df_jan['duration(mins)'] <= 60)].copy()

In [9]:
len(df)/len(df_jan)

0.9827547930522406

### Q4: What's the dimensionality of this matrix (number of columns)?

In [10]:
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,duration(mins)
0,1,2022-01-01 00:35:40,2022-01-01 00:53:29,2.0,3.80,1.0,N,142,236,1,14.5,3.0,0.5,3.65,0.0,0.3,21.95,2.5,0.0,17.816667
1,1,2022-01-01 00:33:43,2022-01-01 00:42:07,1.0,2.10,1.0,N,236,42,1,8.0,0.5,0.5,4.00,0.0,0.3,13.30,0.0,0.0,8.400000
2,2,2022-01-01 00:53:21,2022-01-01 01:02:19,1.0,0.97,1.0,N,166,166,1,7.5,0.5,0.5,1.76,0.0,0.3,10.56,0.0,0.0,8.966667
3,2,2022-01-01 00:25:21,2022-01-01 00:35:23,1.0,1.09,1.0,N,114,68,2,8.0,0.5,0.5,0.00,0.0,0.3,11.80,2.5,0.0,10.033333
4,2,2022-01-01 00:36:48,2022-01-01 01:14:20,1.0,4.30,1.0,N,68,163,1,23.5,0.5,0.5,3.00,0.0,0.3,30.30,2.5,0.0,37.533333


In [11]:
selected_cols = ['PULocationID','DOLocationID']

In [12]:
df[selected_cols] = df[selected_cols].astype(str)

In [13]:

train_dicts = df[selected_cols].to_dict(orient='records')

In [14]:
dv = DictVectorizer()

In [15]:
X_train = dv.fit_transform(train_dicts)

In [16]:
X_train.shape

(2421440, 515)

###  Q5. Training a model

In [17]:
target = 'duration(mins)'
y_train = df[target].values

lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

mean_squared_error(y_train, y_pred, squared=False)

6.986190814952337

###  Q6. Evaluating the model

In [18]:
df_feb = pd.read_parquet("./data/yellow_tripdata_2022-02.parquet")

In [19]:
df_feb['duration(mins)'] = df_feb.progress_apply(lambda x:get_duration(x),axis = 1)

100%|██████████| 2979431/2979431 [00:19<00:00, 153586.12it/s]


In [20]:
df_feb =df_feb[(df_feb['duration(mins)'] >= 1) & (df_feb['duration(mins)'] <= 60)].copy()

In [21]:
df_feb[selected_cols] = df_feb[selected_cols].astype(str)

In [22]:

eval_dicts = df_feb[selected_cols].to_dict(orient='records')

In [23]:
X_eval =  dv.transform(eval_dicts)

In [24]:
y_pred = lr.predict(X_eval)

In [26]:
mean_squared_error(df_feb['duration(mins)'].values, y_pred, squared=False)

7.786407163179794